In [49]:
import requests as re
url = ('https://data.epa.gov.tw/api/v1/aqx_p_322?offset=1&limit=1000&api_key=b21c95e8-ca23-4874-a503-c53093866b0c')
response = re.get(url)
print(response.status_code)
if response.status_code == 200: #系統預設成功會回傳200
    print("下載成功")
    alljson = response.json()


200
下載成功


In [50]:
jsonData = alljson['records']
for item in jsonData:
    print(item)
    print("================================================================================================================")
    

{'SiteId': '2', 'SiteName': '汐止', 'County': '新北市', 'ItemId': '33', 'ItemName': '細懸浮微粒', 'ItemEngName': 'PM2.5', 'ItemUnit': 'μg/m3', 'MonitorDate': '2022-01-06 00:00:00', 'Concentration': '8'}
{'SiteId': '3', 'SiteName': '萬里', 'County': '新北市', 'ItemId': '33', 'ItemName': '細懸浮微粒', 'ItemEngName': 'PM2.5', 'ItemUnit': 'μg/m3', 'MonitorDate': '2022-01-06 00:00:00', 'Concentration': '13'}
{'SiteId': '4', 'SiteName': '新店', 'County': '新北市', 'ItemId': '33', 'ItemName': '細懸浮微粒', 'ItemEngName': 'PM2.5', 'ItemUnit': 'μg/m3', 'MonitorDate': '2022-01-06 00:00:00', 'Concentration': '10'}
{'SiteId': '5', 'SiteName': '土城', 'County': '新北市', 'ItemId': '33', 'ItemName': '細懸浮微粒', 'ItemEngName': 'PM2.5', 'ItemUnit': 'μg/m3', 'MonitorDate': '2022-01-06 00:00:00', 'Concentration': '12'}
{'SiteId': '6', 'SiteName': '板橋', 'County': '新北市', 'ItemId': '33', 'ItemName': '細懸浮微粒', 'ItemEngName': 'PM2.5', 'ItemUnit': 'μg/m3', 'MonitorDate': '2022-01-06 00:00:00', 'Concentration': '10'}
{'SiteId': '7', 'SiteName': '新莊

In [95]:
import sqlalchemy
sqlalchemy.__version__


'1.4.29'

In [105]:
#connect 連線不同資料庫
from sqlalchemy import create_engine
engine = create_engine('sqlite:///pm25.db', echo=True)

In [116]:
#自訂一個類別, 這個類別對應到我們資料庫的資料表
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date


Base = declarative_base()

class pm25(Base):
    __tablename__ = '每日25'
    id = Column(Integer, primary_key=True, autoincrement=True)
    siteid = Column(Integer)
    sitename = Column(String(10), nullable=False)
    county = Column(String(10), nullable=True)#他的預設就是True,所以可以不設
    itemname = Column(String(20))
    itemengname = Column(String(10))
    itemunit = Column(String(10))
    monitordate = Column(Date)
    concentration = Column(Float)
    
    def __repr__(self):
        return "<PM25(sitename='%s', county='%s', monitordate='%s', concentration='%f')>" % (self.sitename, self.county, self.monitordate, self.concentration)  

In [117]:
pm25.__table__

Table('每日25', MetaData(), Column('id', Integer(), table=<每日25>, primary_key=True, nullable=False), Column('siteid', Integer(), table=<每日25>), Column('sitename', String(length=10), table=<每日25>, nullable=False), Column('county', String(length=10), table=<每日25>), Column('itemname', String(length=20), table=<每日25>), Column('itemengname', String(length=10), table=<每日25>), Column('itemunit', String(length=10), table=<每日25>), Column('monitordate', Date(), table=<每日25>), Column('concentration', Float(), table=<每日25>), schema=None)

In [118]:
pm25.metadata.create_all(engine)

2022-01-07 17:15:14,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-07 17:15:14,158 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("每日25")
2022-01-07 17:15:14,160 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-07 17:15:14,162 INFO sqlalchemy.engine.Engine COMMIT


In [119]:
from sqlalchemy.orm import sessionmaker
from datetime import date
Session = sessionmaker(bind=engine)
session = Session()
'''
pm25_1 = pm25(siteid=3, 
            sitename='汐止', 
            county='新北市', 
            itemname='細懸浮粒子', 
            itemengname='PM2.5', 
            itemunit='μg/m3', 
            monitordate=date(2017,3,25), 
            concentration=8
            )
    
session.add(pm25_1)
session.commit()
'''

"\npm25_1 = pm25(siteid=3, \n            sitename='汐止', \n            county='新北市', \n            itemname='細懸浮粒子', \n            itemengname='PM2.5', \n            itemunit='μg/m3', \n            monitordate=date(2017,3,25), \n            concentration=8\n            )\n    \nsession.add(pm25_1)\nsession.commit()\n"

In [120]:
from datetime import datetime

for item in jsonData:
    dateString=item['MonitorDate']
    datetime1 = datetime.strptime(dateString,'%Y-%m-%d %H:%M:%S')
    
    PM25 = pm25(siteid=int(item['SiteId']), 
                  sitename = item['SiteName'], 
                  county = item['County'], 
                  itemname = item['ItemName'], 
                  itemengname = item['ItemEngName'], 
                  itemunit = item['ItemEngName'], 
                  monitordate = datetime1.date(), 
                  concentration = int(item['Concentration'])
                 )
    session.add(PM25)
session.commit()


2022-01-07 17:15:15,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-07 17:15:15,358 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,360 INFO sqlalchemy.engine.Engine [generated in 0.00169s] (2, '汐止', '新北市', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 8.0)
2022-01-07 17:15:15,363 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,371 INFO sqlalchemy.engine.Engine [cached since 0.01329s ago] (3, '萬里', '新北市', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 13.0)
2022-01-07 17:15:15,377 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,379 INFO sqlalchemy.engine.Engine [cached s

2022-01-07 17:15:15,513 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,515 INFO sqlalchemy.engine.Engine [cached since 0.1563s ago] (26, '苗栗', '苗栗縣', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 12.0)
2022-01-07 17:15:15,516 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,518 INFO sqlalchemy.engine.Engine [cached since 0.1596s ago] (27, '三義', '苗栗縣', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 8.0)
2022-01-07 17:15:15,520 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,521 INFO sqlalchemy.engine.Engine [cached since 0.1633s ago] (28, '豐原', '臺中市', '細懸浮微粒', 'PM2.5', 'PM2.5', '202

2022-01-07 17:15:15,701 INFO sqlalchemy.engine.Engine [cached since 0.3432s ago] (50, '鳳山', '高雄市', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 33.0)
2022-01-07 17:15:15,704 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,709 INFO sqlalchemy.engine.Engine [cached since 0.3508s ago] (51, '大寮', '高雄市', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 35.0)
2022-01-07 17:15:15,715 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,717 INFO sqlalchemy.engine.Engine [cached since 0.3588s ago] (52, '林園', '高雄市', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 39.0)
2022-01-07 17:15:15,721 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?

2022-01-07 17:15:15,825 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,826 INFO sqlalchemy.engine.Engine [cached since 0.468s ago] (80, '關山', '臺東縣', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 4.0)
2022-01-07 17:15:15,829 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,831 INFO sqlalchemy.engine.Engine [cached since 0.4729s ago] (83, '麥寮', '雲林縣', '細懸浮微粒', 'PM2.5', 'PM2.5', '2022-01-06', 14.0)
2022-01-07 17:15:15,833 INFO sqlalchemy.engine.Engine INSERT INTO "每日25" (siteid, sitename, county, itemname, itemengname, itemunit, monitordate, concentration) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2022-01-07 17:15:15,835 INFO sqlalchemy.engine.Engine [cached since 0.4765s ago] (84, '富貴角', '新北市', '細懸浮微粒', 'PM2.5', 'PM2.5', '202

In [124]:
pm25_1 = session.query(pm25).filter_by(sitename='三重').first()
pm25_1

InvalidRequestError: Entity namespace for "NULL" has no property "sitename"